# CycleGAN 

In [1]:
from experiment_utils import ExperimentRun
import tensorflow as tf
import matplotlib.pyplot as plt
from models import build_generator, build_discriminator
from dataset import get_dataset
import os
import time, sys

# os.makedirs("results", exist_ok=True)

2025-11-18 23:27:40.609072: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-18 23:27:40.672806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-18 23:27:42.244560: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Hyperparameters

In [2]:
lambda_cyc, lambda_id = 5.0, 2.5
img_size = 256
batch_size = 2
num_epochs = 100
trainX_pattern = '../../data/animals_balanced/butterfly/*.JPEG'
trainY_pattern = '../../data/origami_images/butterfly/*.jpg'

### Generator & Discriminator

In [3]:
G = build_generator(image_size=img_size, n_blocks=9)
F = build_generator(image_size=img_size, n_blocks=9)
D_X = build_discriminator(image_size=img_size)
D_Y = build_discriminator(image_size=img_size)

I0000 00:00:1763508488.867242    8378 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5



### Optimizer & Loss

In [4]:
# ----------------------------
# 2️⃣ Optimizer & Loss
# ----------------------------
G_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_X_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_Y_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

mse = tf.keras.losses.MeanSquaredError()
L1 = lambda a, b: tf.reduce_mean(tf.abs(a - b))

### Setup for Experiments

In [5]:
#expereiment setup
params = {
    "lambda_cyc": lambda_cyc,
    "lambda_id": lambda_id,
    "img_size": img_size,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "G_lr": 1e-4,
    "D_lr": 1e-4,
    "n_blocks": 9,
}

config = {
    **params,
    "trainX_pattern": trainX_pattern,
    "trainY_pattern": trainY_pattern,
}

exp = ExperimentRun(params=params, config=config)
logger = exp.get_logger()

# connect checkpoint manager
ckpt, ckpt_manager = exp.create_checkpoint_manager(
    G=G, F=F, D_X=D_X, D_Y=D_Y,
    G_optimizer=G_optimizer,
    D_X_optimizer=D_X_optimizer,
    D_Y_optimizer=D_Y_optimizer,
)

2025-11-18 23:28:16,972 - INFO - Experiment directory: experiments/run_20251118-232816


### Training Step

In [6]:
# ----------------------------
# 3️⃣ 1 training step
# ----------------------------
@tf.function
def train_step(real_x, real_y, lambda_cyc, lambda_id):
    with tf.GradientTape(persistent=True) as tape:

        fake_y = G(real_x, training=True)
        fake_x = F(real_y, training=True)

        cyc_x = F(fake_y, training=True)
        cyc_y = G(fake_x, training=True)

        same_x = F(real_x, training=True)
        same_y = G(real_y, training=True)

        D_X_real = D_X(real_x, training=True)
        D_X_fake = D_X(fake_x, training=True)
        D_Y_real = D_Y(real_y, training=True)
        D_Y_fake = D_Y(fake_y, training=True)

        G_GAN_loss = mse(tf.ones_like(D_Y_fake), D_Y_fake)
        F_GAN_loss = mse(tf.ones_like(D_X_fake), D_X_fake)

        D_X_loss = 0.5 * (mse(tf.ones_like(D_X_real), D_X_real) + mse(tf.zeros_like(D_X_fake), D_X_fake))
        D_Y_loss = 0.5 * (mse(tf.ones_like(D_Y_real), D_Y_real) + mse(tf.zeros_like(D_Y_fake), D_Y_fake))

        cycle_loss = L1(cyc_x, real_x) + L1(cyc_y, real_y)
        id_loss = L1(same_x, real_x) + L1(same_y, real_y)

        G_total_loss = G_GAN_loss + F_GAN_loss + lambda_cyc * cycle_loss + lambda_id * id_loss

    G_grads = tape.gradient(G_total_loss, G.trainable_variables + F.trainable_variables)
    D_X_grads = tape.gradient(D_X_loss, D_X.trainable_variables)
    D_Y_grads = tape.gradient(D_Y_loss, D_Y.trainable_variables)

    del tape
    G_optimizer.apply_gradients(zip(G_grads, G.trainable_variables + F.trainable_variables))
    D_X_optimizer.apply_gradients(zip(D_X_grads, D_X.trainable_variables))
    D_Y_optimizer.apply_gradients(zip(D_Y_grads, D_Y.trainable_variables))

    return G_total_loss, D_X_loss, D_Y_loss


### Load Dataset

In [7]:
trainX = get_dataset(trainX_pattern, batch_size=batch_size, img_size=img_size, augment=True)
trainY = get_dataset(trainY_pattern, batch_size=batch_size, img_size=img_size, augment=True)

### Training Cycle

In [8]:
cardX = tf.data.experimental.cardinality(trainX).numpy()
cardY = tf.data.experimental.cardinality(trainY).numpy()
num_steps_per_epoch = int(min(cardX, cardY))
bar_len = 30
ckpt_interval = 5

for epoch in range(num_epochs):
    
    epoch_start = time.time() 
    
    logger.info(f"Epoch {epoch+1}/{num_epochs}")
    step = 0
    
    for real_x, real_y in zip(trainX, trainY):
        step += 1
        G_loss, DX_loss, DY_loss = train_step(real_x, real_y, lambda_cyc, lambda_id)
        
        bar = exp.progress_bar(step, num_steps_per_epoch, bar_len=bar_len)
        sys.stdout.write(
            "\r"
            f"Epoch {epoch+1}/{num_epochs} "
            f"{bar}  step {step}/{num_steps_per_epoch}"
        )
        sys.stdout.flush()
        
        # if step % 50 == 0:
        #     logger.info(
        #         f"  step {step}: G={float(G_loss.numpy()):.3f} "
        #         f"DX={float(DX_loss.numpy()):.3f} DY={float(DY_loss.numpy()):.3f}"
        #     )
        loss_dict = {
            "G_total": G_loss,
            "D_X": DX_loss,
            "D_Y": DY_loss,
        }
        exp.log_losses(epoch + 1, step, loss_dict)
        
    sys.stdout.write("\n")
    sys.stdout.flush()
    
    epoch_time = time.time() - epoch_start
    time_per_step = epoch_time / step

    logger.info(
        f"Epoch {epoch+1} completed in {epoch_time:.2f}s "
        f"({time_per_step:.4f}s per step)"
    )
    
    if (epoch + 1) % ckpt_interval == 0 or (epoch + 1) == num_epochs:
        exp.save_checkpoint()

    sample = next(iter(trainX))
    fake_y = G(sample, training=False)

    if (epoch + 1) % 10 == 0:
        logger.info(
            "Epoch %d: G=%.3f DX=%.3f DY=%.3f",
            epoch + 1, float(G_loss.numpy()), float(DX_loss.numpy()), float(DY_loss.numpy())
        )
        sample = next(iter(trainX))
        fake_y = G(sample, training=False)
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1); plt.imshow((sample[0] + 1) / 2); plt.title("Real X"); plt.axis("off")
        plt.subplot(1, 2, 2); plt.imshow((fake_y[0] + 1) / 2); plt.title("Fake Y"); plt.axis("off")
        plt.tight_layout()
        out_path = exp.results_dir / f"epoch_{epoch+1}.png"
        plt.savefig(out_path)
        plt.close()
        
print("\nGenerating final comparison on test image...")



2025-11-18 23:28:22,225 - INFO - Epoch 1/100
2025-11-18 23:29:10.741289: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90501


Epoch 1/100 [============================..]  96.2%  step 50/52

2025-11-18 23:30:17,837 - INFO -   step 50: G=6.412 DX=0.347 DY=0.502


Epoch 1/100 [==============================] 100.0%  step 52/52

2025-11-18 23:30:46.229765: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Epoch 23/100 [============================..]  96.2%  step 50/52

2025-11-18 23:56:19,621 - INFO -   step 50: G=5.143 DX=0.178 DY=0.310
INFO:run_20251118-232816:  step 50: G=5.143 DX=0.178 DY=0.310


Epoch 23/100 [==============================] 100.0%  step 52/52


2025-11-18 23:56:22,362 - INFO - Epoch 23 completed in 67.64s (1.3008s per step)
INFO:run_20251118-232816:Epoch 23 completed in 67.64s (1.3008s per step)
2025-11-18 23:56:23.153454: W tensorflow/core/framework/op_kernel.cc:1855] OP_REQUIRES failed at save_restore_v2_ops.cc:141 : RESOURCE_EXHAUSTED: experiments/run_20251118-232816/checkpoints/ckpt-23_temp/part-00000-of-00001.data-00000-of-00001.tempstate6630626494692604353; No space left on device


ResourceExhaustedError: {{function_node __wrapped__SaveV2_dtypes_668_device_/job:localhost/replica:0/task:0/device:CPU:0}} experiments/run_20251118-232816/checkpoints/ckpt-23_temp/part-00000-of-00001.data-00000-of-00001.tempstate6630626494692604353; No space left on device [Op:SaveV2]

In [ ]:
test_path = "data/NST_butterfly.jpg"
if os.path.exists(test_path):
    test_img = tf.io.read_file(test_path)
    test_img = tf.image.decode_jpeg(test_img, channels=3)
    test_img = tf.image.resize(test_img, [img_size, img_size])
    test_img = (tf.cast(test_img, tf.float32) / 127.5) - 1.0
    test_img = tf.expand_dims(test_img, 0)

    fake_y = G(test_img, training=False)
    fake_y = (fake_y[0] + 1.0) / 2.0

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1); plt.imshow((test_img[0] + 1) / 2.0); plt.title("Original Image"); plt.axis("off")
    plt.subplot(1, 2, 2); plt.imshow(fake_y); plt.title("CycleGAN Output"); plt.axis("off")
    plt.tight_layout()
    plt.savefig("results/final_comparison.png")
    plt.close()
    print("✅ Final comparison saved at results/final_comparison.png")
else:
    print("⚠️ Skipped final comparison: test image not found at data/single_eval/test_img.jpg")


In [18]:
import os
print("CWD =", os.getcwd())


CWD = /home/ubuntu/projects/Origami-Model-using-CycleGAN/CycleGAN


In [19]:
import glob

print("X files:", len(glob.glob(trainX_pattern)))
print("Y files:", len(glob.glob(trainY_pattern)))

print("Some X:", glob.glob(trainX_pattern)[:3])
print("Some Y:", glob.glob(trainY_pattern)[:3])


X files: 0
Y files: 0
Some X: []
Some Y: []
